This noteboook is a ML classifier for fake news

Import and combine the data

In [1]:
import pandas as pd

In [13]:
# Load datasets
fake = pd.read_csv("data/fake.csv")
true = pd.read_csv("data/True.csv")
# Add labels: 0 for true, 1 for fake
fake['label'] = 1
true['label'] = 0

In [ ]:
# Display information about the datasets
fake.info()
true.info()

In [15]:
# Combine the datasets
data = pd.concat([fake, true], ignore_index=True)

In [ ]:
# Shuffle the data for randomness
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the structure
print(data.head())

Preprocess and Clean the data

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Basic text cleaning function
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text


In [ ]:
# Apply text cleaning
data['text'] = data['text'].apply(clean_text)
data.head()

In [ ]:
# Split features and labels
X = data['text']
y = data['label']

In [20]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

3. Train a Machine Learning Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Train a Naive Bayes model
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Experiment with Other Models

In [42]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
# Tokenize text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=200)
X_test_pad = pad_sequences(X_test_seq, maxlen=200)

# Build an LSTM model
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=200),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=3, batch_size=64)

# Evaluate
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Accuracy: {accuracy}")

ModuleNotFoundError: No module named 'keras.preprocessing.text'

In [40]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout


import tensorflow as tf
print(tf.__version__)

print("#############")
import tensorflow.keras
print(tensorflow.keras.__version__)

2.17.0
#############


ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout



In [41]:
import tensorflow as tf
from keras.layers import Embedding, LSTM, Dense, Dropout
print(tf.__version__)
print(Embedding)
print(LSTM)
print(Dense)
print(Dropout)


2.17.0
<class 'keras.src.layers.core.embedding.Embedding'>
<class 'keras.src.layers.rnn.lstm.LSTM'>
<class 'keras.src.layers.core.dense.Dense'>
<class 'keras.src.layers.regularization.dropout.Dropout'>
